In [1]:
from transformers import AutoFeatureExtractor, DeiTForImageClassificationWithTeacher, DeiTConfig
from PIL import Image
import requests
import json
from ipynb.fs.full.read_in_data import read_in_data
import torch
from typing import Optional, Set, Tuple, Union

/home/aleksandra/Documents/hackathon/knum_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("../public_dataset/reference_images_part1.json") as f:
    ref_json = json.load(f)

dir_str = "../public_dataset/reference_images_part1"
data = read_in_data(dir_str, ref_json)

data.head()

,id,image_id,bbox,category_id,img
12,12,4,"[665, 448, 206, 174]",8,<PIL.Image.Image image mode=RGBA size=206x174 ...
13,13,4,"[475, 186, 236, 186]",8,<PIL.Image.Image image mode=RGBA size=236x186 ...
14,14,4,"[962, 76, 229, 217]",8,<PIL.Image.Image image mode=RGBA size=229x217 ...
15,15,4,"[778, 250, 163, 135]",8,<PIL.Image.Image image mode=RGBA size=163x135 ...
16,16,4,"[184, 377, 244, 194]",8,<PIL.Image.Image image mode=RGBA size=244x194 ...


In [3]:
class DeiTForImageClassificationWithTeacher(DeiTForImageClassificationWithTeacher):
    def __init__(self, config: DeiTConfig) -> None:
        super().__init__(config)

    def forward(
        self,
        pixel_values: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.deit(
            pixel_values,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        return outputs[0][:, 0, :]

In [4]:
image = data.img[12]

image_ = Image.new("RGB", image.size, (255, 255, 255))
image_.paste(image, mask=image.split()[3]) 



feature_extractor = AutoFeatureExtractor.from_pretrained('facebook/deit-small-distilled-patch16-224')
model = DeiTForImageClassificationWithTeacher.from_pretrained('facebook/deit-small-distilled-patch16-224')
inputs = feature_extractor(images=image_, return_tensors="pt")
outputs = model(**inputs)
print(outputs.shape) # co to jest 198??

torch.Size([1, 384])
